Scrape <a href="https://www.berlin.de/polizei/polizeimeldungen">berlin.de/polizei/polizeimeldungen</a> , um alle zu einem bestimmten Stichtag vorhandenen Meldungen zu bekommen


Aus https://www.berlin.de/robots.txt:

"Any automated program to retrieve information from our public websites is only allowed subject to the following restrictions:

* It must clearly and uniquely identify itself in the HTTP User-Agent header.
 https://webmasters.stackexchange.com/questions/6205/what-user-agent-should-i-set"


In [68]:
import requests 
import lxml.html 
import json

In [38]:
#html = requests.get("https://www.berlin.de/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=1")
#doc = lxml.html.fromstring(html.content)

In [69]:
def get_archives():
    url = "https://www.berlin.de/polizei/polizeimeldungen/archiv/"
    response = requests.get(url)
    doc = lxml.html.fromstring(response.content)
    links_to_archives = doc.xpath("//div[@class = 'html5-section article']//div[@class = 'html5-section block modul-text_bild']/div[@class = 'html5-section body']//div[@class = 'textile']//a/@href")
    return links_to_archives

In [40]:
#get_archives()

['/polizei/polizeimeldungen/archiv/2019/',
 '/polizei/polizeimeldungen/archiv/2018/',
 '/polizei/polizeimeldungen/archiv/2017/',
 '/polizei/polizeimeldungen/archiv/2016/',
 '/polizei/polizeimeldungen/archiv/2015/',
 '/polizei/polizeimeldungen/archiv/2014/']

In [70]:
def get_links_to_reports(archive_site):
    url = "https://www.berlin.de" + archive_site
    response = requests.get(url)
    doc = lxml.html.fromstring(response.content)
    links_to_reports = doc.xpath("//div[contains(@class, 'cell')]/a/@href")
    return links_to_reports    

In [42]:
#get_links_to_reports('/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=1')

['/polizei/polizeimeldungen/pressemitteilung.350011.php',
 '/polizei/polizeimeldungen/pressemitteilung.429288.php',
 '/polizei/polizeimeldungen/pressemitteilung.429287.php',
 '/polizei/polizeimeldungen/pressemitteilung.429277.php',
 '/polizei/polizeimeldungen/pressemitteilung.429276.php',
 '/polizei/polizeimeldungen/pressemitteilung.429268.php',
 '/polizei/polizeimeldungen/pressemitteilung.429267.php',
 '/polizei/polizeimeldungen/pressemitteilung.429266.php',
 '/polizei/polizeimeldungen/pressemitteilung.429159.php',
 '/polizei/polizeimeldungen/pressemitteilung.429125.php',
 '/polizei/polizeimeldungen/pressemitteilung.427101.php',
 '/polizei/polizeimeldungen/pressemitteilung.429091.php',
 '/polizei/polizeimeldungen/pressemitteilung.379061.php',
 '/polizei/polizeimeldungen/pressemitteilung.428925.php',
 '/polizei/polizeimeldungen/pressemitteilung.428923.php',
 '/polizei/polizeimeldungen/pressemitteilung.428922.php',
 '/polizei/polizeimeldungen/pressemitteilung.428921.php',
 '/polizei/pol

In [43]:
# titles = doc.xpath("//div[contains(@class, 'cell')]//a/text()")

In [44]:
# print(titles)

Die gesammelten Links im Weiteren itererieren, um auf verlinkte Inhalte (Polizeimeldungstexte) zuzugreifen

In [45]:
#links = doc.xpath("//div[contains(@class, 'cell')]/a/@href")

#links = links[0:10]
#print(links)

In [46]:
# %run -i 'polizei_berlin_scraper.py' links /home/rosanna/Dokumente/semantischeSuche/jsons

Sind das die Zeiten der Herausgabe der Polizeimeldungen oder des Vorfalls? Vermutlich ersteres

In [47]:
# dates = doc.xpath("//div[contains(@class, 'cell')]/text()")
# print(dates[:10])

Orte in Linkliste sind in Form der 12 Bezirke Berlins plus Kategorie "bezirksübergreifend" angegeben:

In [48]:
# places = doc.xpath("//span[contains(@class, 'category')]/text()")

# print(places)
# places_as_set = set(places)
# print(places_as_set)

In [49]:
"""
def get_text(link):
    url = "https://www.berlin.de" + link
    html2 = requests.get(url)
    doc2 = lxml.html.fromstring(html2.content)
    article_headline = doc2.xpath("//h1[@class = 'title']/text()")
    article_time_place = doc2.xpath("//div[@class = 'polizeimeldung']/text()")
    article_subheads = doc2.xpath("//div[@class = 'textile']/p/strong/text()")
   # article_paragraphs = doc2.xpath("//div[@class = 'textile']/p/text()")
    article_paragraphs_inclu_caps = doc2.xpath("//div[@class = 'textile']/p/text() | //div[@class = 'textile']//span[@class = 'caps']/text()")
   
    return article_headline, article_time_place, article_subheads, article_paragraphs_inclu_caps, url
    """

'\ndef get_text(link):\n    url = "https://www.berlin.de" + link\n    html2 = requests.get(url)\n    doc2 = lxml.html.fromstring(html2.content)\n    article_headline = doc2.xpath("//h1[@class = \'title\']/text()")\n    article_time_place = doc2.xpath("//div[@class = \'polizeimeldung\']/text()")\n    article_subheads = doc2.xpath("//div[@class = \'textile\']/p/strong/text()")\n   # article_paragraphs = doc2.xpath("//div[@class = \'textile\']/p/text()")\n    article_paragraphs_inclu_caps = doc2.xpath("//div[@class = \'textile\']/p/text() | //div[@class = \'textile\']//span[@class = \'caps\']/text()")\n   \n    return article_headline, article_time_place, article_subheads, article_paragraphs_inclu_caps, url\n    '

In [71]:
def get_dict_from_scraped(source):
    url = "https://www.berlin.de" + source
    html2 = requests.get(url)
    doc2 = lxml.html.fromstring(html2.content)
    
    response = html2.status_code
    
    article_headline = doc2.xpath("//h1[@class = 'title']/text()")
    article_time_place = doc2.xpath("//div[@class = 'polizeimeldung']/text()")
    article_subheads = doc2.xpath("//div[@class = 'textile']/p/strong/text()")
    article_paragraphs_inclu_caps = doc2.xpath("//div[@class = 'textile']/p/text() | //div[@class = 'textile']//span[@class = 'caps']/text()")
    
    published = article_time_place[0].strip() if article_time_place else ''
  #  bezirk = article_time_place[1].strip() if article_time_place[1] else ''
    bezirk = article_time_place[1].strip() if (len(article_time_place) > 1) else ''
    article = "".join(article_paragraphs_inclu_caps).strip("\n ")
    head = "".join(article_headline)
    subs = "".join(article_subheads)

    polizei_data = {
        "response":response,
        "headline":head,
        "published":published,
        "bezirk":bezirk,
        "subheads":subs,
        "article":article,
        "url":url
        }
    
    return polizei_data

In [51]:
"""
def to_json(scraped_data, local_path, index):
    #with open('%s/%s-summary.json' % (local_path, index),'w') as poldat:
        #json.dump(scraped_data,poldat,indent = 4,ensure_ascii=False)
    with open('%s/%s-summary.json' % (local_path, index),'w') as poldat:
        json.dump(scraped_data,poldat,indent = 4,ensure_ascii=False)
        """

"\ndef to_json(scraped_data, local_path, index):\n    #with open('%s/%s-summary.json' % (local_path, index),'w') as poldat:\n        #json.dump(scraped_data,poldat,indent = 4,ensure_ascii=False)\n    with open('%s/%s-summary.json' % (local_path, index),'w') as poldat:\n        json.dump(scraped_data,poldat,indent = 4,ensure_ascii=False)\n        "

In [72]:
def to_json3(dict):
    jsoni = json.dumps(dict)
    f = open("dict_polizei.json","w")
    f.write(jsoni)
    f.close()

In [53]:
"""
def to_json2(scraped_data):
   # with open('%s/%s-summary.json' % (local_path, index),'w') as poldat:
    #jso = json.dumps(scraped_data)
    return json.dumps(scraped_data)
    """

"\ndef to_json2(scraped_data):\n   # with open('%s/%s-summary.json' % (local_path, index),'w') as poldat:\n    #jso = json.dumps(scraped_data)\n    return json.dumps(scraped_data)\n    "

Alle Links (Liste von URLs) iterieren. Daten in Liste von Dicts packen und als json-file speichern.

In [73]:
def get_data_from_url_list(links):
    
    scraped_list_of_dicts = []
    
    for link in links:
        data = get_dict_from_scraped(link)
       # to_json(data, "/home/rosanna/Dokumente/semantischeSuche/jsons", links.index(link))
       # json_obj = to_json2(data)
       # json_obj = json.dumps(data,indent = 4,ensure_ascii=False)
    
        scraped_list_of_dicts.append(data)
    
    #print(scraped_list_of_dicts)
    test_dict = {"items":scraped_list_of_dicts}
    #print(type(test_dict))
    print(test_dict)
    to_json3(test_dict)
    #to_json(test_dict, "/home/rosanna/Dokumente/semantischeSuche", 13)

Links zu einzelne Archivseiten holen (diese enthalten jeweils eine Liste von max. 50 Links zu Polizeimeldungen). Archivmeldungen seit 2014 bis heute verfügbar. param arch_link e.g. "/polizei/polizeimeldungen/archiv/2015/"

In [74]:
def get_links_per_archive_site(arch_link):
    url = "https://www.berlin.de" + arch_link
    html = requests.get(url)
    doc = lxml.html.fromstring(html.content)
    last_site = doc.xpath("//li[@class = 'pager-item last']/a/text()")[0]
    links_per_archive = []
    for i in range(1, int(last_site)+1):
        lin = arch_link + "?page_at_1_0=" + str(i)
        links_per_archive.append(lin)
        
    return links_per_archive

In [58]:
# get_links_per_archive_site("/polizei/polizeimeldungen/archiv/2015/")

['/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=1',
 '/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=2',
 '/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=3',
 '/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=4',
 '/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=5',
 '/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=6',
 '/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=7',
 '/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=8',
 '/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=9',
 '/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=10',
 '/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=11',
 '/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=12',
 '/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=13',
 '/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=14',
 '/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=15',
 '/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=16',
 '/polizei/polizeimeldungen/archiv/2015/?page_at_1_0=17',
 '/polizei/polizeimeldu

In [75]:
archives = get_archives()
print(archives)

['/polizei/polizeimeldungen/archiv/2019/', '/polizei/polizeimeldungen/archiv/2018/', '/polizei/polizeimeldungen/archiv/2017/', '/polizei/polizeimeldungen/archiv/2016/', '/polizei/polizeimeldungen/archiv/2015/', '/polizei/polizeimeldungen/archiv/2014/']


In [78]:
archive_sites = []

for archive in archives:
    arch_links = get_links_per_archive_site(archive)
    archive_sites.extend(arch_links)
    
print(archive_sites)

# holte Archivseiten als Liste in dieser Form: ['/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=1', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=2', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=3', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=4', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=5', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=6', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=7', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=8', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=9', '/polizei/polizeimeldungen/archiv/2019/?page_at_

['/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=1', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=2', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=3', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=4', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=5', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=6', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=7', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=8', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=9', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=10', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=11', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=12', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=13', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=14', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=15', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=16', '/polizei/polizeimeldungen/archiv/2019/?page_at_1_0=17', '/polizei/polizeimeldungen/archiv/2019/

In [79]:
all_links_to_reports = []

for archive_site in archive_sites:
    links_to_reports = get_links_to_reports(archive_site)
    print(links_to_reports)
    all_links_to_reports.extend(links_to_reports)
    print(len(all_links_to_reports))
    
print(all_links_to_reports)
#len(all_links_to_reports)

# hat mir über 7000 Links gebracht. Was ist mit den fehlenden über 3000? wegen 429 nicht bekommen?

['/polizei/polizeimeldungen/pressemitteilung.862252.php', '/polizei/polizeimeldungen/pressemitteilung.861997.php', '/polizei/polizeimeldungen/pressemitteilung.861992.php', '/polizei/polizeimeldungen/pressemitteilung.861991.php', '/polizei/polizeimeldungen/pressemitteilung.861988.php', '/polizei/polizeimeldungen/pressemitteilung.861986.php', '/polizei/polizeimeldungen/pressemitteilung.861471.php', '/polizei/polizeimeldungen/pressemitteilung.861359.php', '/polizei/polizeimeldungen/pressemitteilung.861354.php', '/polizei/polizeimeldungen/pressemitteilung.861351.php', '/polizei/polizeimeldungen/pressemitteilung.861347.php', '/polizei/polizeimeldungen/pressemitteilung.860961.php', '/polizei/polizeimeldungen/pressemitteilung.860958.php', '/polizei/polizeimeldungen/pressemitteilung.860957.php', '/polizei/polizeimeldungen/pressemitteilung.860954.php', '/polizei/polizeimeldungen/pressemitteilung.859723.php', '/polizei/polizeimeldungen/pressemitteilung.860932.php', '/polizei/polizeimeldungen/pre

['/polizei/polizeimeldungen/pressemitteilung.853967.php', '/polizei/polizeimeldungen/pressemitteilung.853964.php', '/polizei/polizeimeldungen/pressemitteilung.853937.php', '/polizei/polizeimeldungen/pressemitteilung.853789.php', '/polizei/polizeimeldungen/pressemitteilung.853137.php', '/polizei/polizeimeldungen/pressemitteilung.853126.php', '/polizei/polizeimeldungen/pressemitteilung.853123.php', '/polizei/polizeimeldungen/pressemitteilung.852957.php', '/polizei/polizeimeldungen/pressemitteilung.852751.php', '/polizei/polizeimeldungen/pressemitteilung.852750.php', '/polizei/polizeimeldungen/pressemitteilung.852685.php', '/polizei/polizeimeldungen/pressemitteilung.852663.php', '/polizei/polizeimeldungen/pressemitteilung.852662.php', '/polizei/polizeimeldungen/pressemitteilung.851115.php', '/polizei/polizeimeldungen/pressemitteilung.852445.php', '/polizei/polizeimeldungen/pressemitteilung.852442.php', '/polizei/polizeimeldungen/pressemitteilung.852438.php', '/polizei/polizeimeldungen/pre

['/polizei/polizeimeldungen/pressemitteilung.840401.php', '/polizei/polizeimeldungen/pressemitteilung.840400.php', '/polizei/polizeimeldungen/pressemitteilung.840398.php', '/polizei/polizeimeldungen/pressemitteilung.840346.php', '/polizei/polizeimeldungen/pressemitteilung.840309.php', '/polizei/polizeimeldungen/pressemitteilung.840308.php', '/polizei/polizeimeldungen/pressemitteilung.840307.php', '/polizei/polizeimeldungen/pressemitteilung.840306.php', '/polizei/polizeimeldungen/pressemitteilung.840305.php', '/polizei/polizeimeldungen/pressemitteilung.840304.php', '/polizei/polizeimeldungen/pressemitteilung.840303.php', '/polizei/polizeimeldungen/pressemitteilung.840292.php', '/polizei/polizeimeldungen/pressemitteilung.840290.php', '/polizei/polizeimeldungen/pressemitteilung.840289.php', '/polizei/polizeimeldungen/pressemitteilung.840288.php', '/polizei/polizeimeldungen/pressemitteilung.840284.php', '/polizei/polizeimeldungen/pressemitteilung.840059.php', '/polizei/polizeimeldungen/pre

['/polizei/polizeimeldungen/pressemitteilung.829251.php', '/polizei/polizeimeldungen/pressemitteilung.829250.php', '/polizei/polizeimeldungen/pressemitteilung.829246.php', '/polizei/polizeimeldungen/pressemitteilung.829211.php', '/polizei/polizeimeldungen/pressemitteilung.829197.php', '/polizei/polizeimeldungen/pressemitteilung.829195.php', '/polizei/polizeimeldungen/pressemitteilung.827634.php', '/polizei/polizeimeldungen/pressemitteilung.828988.php', '/polizei/polizeimeldungen/pressemitteilung.828942.php', '/polizei/polizeimeldungen/pressemitteilung.828897.php', '/polizei/polizeimeldungen/pressemitteilung.828896.php', '/polizei/polizeimeldungen/pressemitteilung.828862.php', '/polizei/polizeimeldungen/pressemitteilung.828861.php', '/polizei/polizeimeldungen/pressemitteilung.828857.php', '/polizei/polizeimeldungen/pressemitteilung.828856.php', '/polizei/polizeimeldungen/pressemitteilung.828852.php', '/polizei/polizeimeldungen/pressemitteilung.828855.php', '/polizei/polizeimeldungen/pre

['/polizei/polizeimeldungen/pressemitteilung.818808.php', '/polizei/polizeimeldungen/pressemitteilung.818790.php', '/polizei/polizeimeldungen/pressemitteilung.818416.php', '/polizei/polizeimeldungen/pressemitteilung.818683.php', '/polizei/polizeimeldungen/pressemitteilung.818467.php', '/polizei/polizeimeldungen/pressemitteilung.818462.php', '/polizei/polizeimeldungen/pressemitteilung.818442.php', '/polizei/polizeimeldungen/pressemitteilung.818441.php', '/polizei/polizeimeldungen/pressemitteilung.818440.php', '/polizei/polizeimeldungen/pressemitteilung.818439.php', '/polizei/polizeimeldungen/pressemitteilung.818438.php', '/polizei/polizeimeldungen/pressemitteilung.818437.php', '/polizei/polizeimeldungen/pressemitteilung.818436.php', '/polizei/polizeimeldungen/pressemitteilung.818435.php', '/polizei/polizeimeldungen/pressemitteilung.818431.php', '/polizei/polizeimeldungen/pressemitteilung.818430.php', '/polizei/polizeimeldungen/pressemitteilung.818428.php', '/polizei/polizeimeldungen/pre

...

['/polizei/polizeimeldungen/pressemitteilung.288251.php', '/polizei/polizeimeldungen/pressemitteilung.288560.php', '/polizei/polizeimeldungen/pressemitteilung.288417.php', '/polizei/polizeimeldungen/pressemitteilung.287911.php', '/polizei/polizeimeldungen/pressemitteilung.287910.php', '/polizei/polizeimeldungen/pressemitteilung.287909.php', '/polizei/polizeimeldungen/pressemitteilung.287908.php', '/polizei/polizeimeldungen/pressemitteilung.287907.php', '/polizei/polizeimeldungen/pressemitteilung.287906.php', '/polizei/polizeimeldungen/pressemitteilung.287905.php', '/polizei/polizeimeldungen/pressemitteilung.287903.php', '/polizei/polizeimeldungen/pressemitteilung.287902.php', '/polizei/polizeimeldungen/pressemitteilung.287901.php', '/polizei/polizeimeldungen/pressemitteilung.287900.php', '/polizei/polizeimeldungen/pressemitteilung.287899.php', '/polizei/polizeimeldungen/pressemitteilung.287898.php', '/polizei/polizeimeldungen/pressemitteilung.287897.php', '/polizei/polizeimeldungen/pre

['/polizei/polizeimeldungen/pressemitteilung.265416.php', '/polizei/polizeimeldungen/pressemitteilung.265415.php', '/polizei/polizeimeldungen/pressemitteilung.265414.php', '/polizei/polizeimeldungen/pressemitteilung.265413.php', '/polizei/polizeimeldungen/pressemitteilung.265412.php', '/polizei/polizeimeldungen/pressemitteilung.265410.php', '/polizei/polizeimeldungen/pressemitteilung.265409.php', '/polizei/polizeimeldungen/pressemitteilung.265408.php', '/polizei/polizeimeldungen/pressemitteilung.265407.php', '/polizei/polizeimeldungen/pressemitteilung.265364.php', '/polizei/polizeimeldungen/pressemitteilung.265324.php', '/polizei/polizeimeldungen/pressemitteilung.265252.php', '/polizei/polizeimeldungen/pressemitteilung.265053.php', '/polizei/polizeimeldungen/pressemitteilung.217628.php', '/polizei/polizeimeldungen/pressemitteilung.264873.php', '/polizei/polizeimeldungen/pressemitteilung.264872.php', '/polizei/polizeimeldungen/pressemitteilung.264871.php', '/polizei/polizeimeldungen/pre

['/polizei/polizeimeldungen/pressemitteilung.247356.php', '/polizei/polizeimeldungen/pressemitteilung.247355.php', '/polizei/polizeimeldungen/pressemitteilung.247354.php', '/polizei/polizeimeldungen/pressemitteilung.247353.php', '/polizei/polizeimeldungen/pressemitteilung.247352.php', '/polizei/polizeimeldungen/pressemitteilung.247296.php', '/polizei/polizeimeldungen/pressemitteilung.247237.php', '/polizei/polizeimeldungen/pressemitteilung.247246.php', '/polizei/polizeimeldungen/pressemitteilung.247245.php', '/polizei/polizeimeldungen/pressemitteilung.247244.php', '/polizei/polizeimeldungen/pressemitteilung.247243.php', '/polizei/polizeimeldungen/pressemitteilung.247242.php', '/polizei/polizeimeldungen/pressemitteilung.247153.php', '/polizei/polizeimeldungen/pressemitteilung.247142.php', '/polizei/polizeimeldungen/pressemitteilung.247135.php', '/polizei/polizeimeldungen/pressemitteilung.247089.php', '/polizei/polizeimeldungen/pressemitteilung.247009.php', '/polizei/polizeimeldungen/pre

['/polizei/polizeimeldungen/pressemitteilung.228486.php', '/polizei/polizeimeldungen/pressemitteilung.228480.php', '/polizei/polizeimeldungen/pressemitteilung.228477.php', '/polizei/polizeimeldungen/pressemitteilung.228474.php', '/polizei/polizeimeldungen/pressemitteilung.228347.php', '/polizei/polizeimeldungen/pressemitteilung.228334.php', '/polizei/polizeimeldungen/pressemitteilung.228321.php', '/polizei/polizeimeldungen/pressemitteilung.228314.php', '/polizei/polizeimeldungen/pressemitteilung.228180.php', '/polizei/polizeimeldungen/pressemitteilung.227919.php', '/polizei/polizeimeldungen/pressemitteilung.227917.php', '/polizei/polizeimeldungen/pressemitteilung.227915.php', '/polizei/polizeimeldungen/pressemitteilung.227913.php', '/polizei/polizeimeldungen/pressemitteilung.227912.php', '/polizei/polizeimeldungen/pressemitteilung.226758.php', '/polizei/polizeimeldungen/pressemitteilung.226713.php', '/polizei/polizeimeldungen/pressemitteilung.226601.php', '/polizei/polizeimeldungen/pre

['/polizei/polizeimeldungen/pressemitteilung.192244.php', '/polizei/polizeimeldungen/pressemitteilung.191651.php', '/polizei/polizeimeldungen/pressemitteilung.191601.php', '/polizei/polizeimeldungen/pressemitteilung.190663.php', '/polizei/polizeimeldungen/pressemitteilung.190434.php', '/polizei/polizeimeldungen/pressemitteilung.190430.php', '/polizei/polizeimeldungen/pressemitteilung.190427.php', '/polizei/polizeimeldungen/pressemitteilung.190424.php', '/polizei/polizeimeldungen/pressemitteilung.190420.php', '/polizei/polizeimeldungen/pressemitteilung.190155.php', '/polizei/polizeimeldungen/pressemitteilung.190131.php', '/polizei/polizeimeldungen/pressemitteilung.190101.php', '/polizei/polizeimeldungen/pressemitteilung.190073.php', '/polizei/polizeimeldungen/pressemitteilung.187951.php', '/polizei/polizeimeldungen/pressemitteilung.187848.php', '/polizei/polizeimeldungen/pressemitteilung.187805.php', '/polizei/polizeimeldungen/pressemitteilung.187685.php', '/polizei/polizeimeldungen/pre

In [80]:

get_data_from_url_list(all_links_to_reports)
# versucht das gesamte Archiv zu scrapen. Lief bei mir 25 Minuten, dann habe ich es abgebrochen. Besser nicht machen..

KeyboardInterrupt: 